#### 1. Importing Data & Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Para encontrar a morada de cada
import reverse_geocoder as rg
from geopy.geocoders import Nominatim
from multiprocessing import Pool

#Para fazer mapas
import folium
from folium.plugins import HeatMap, MarkerCluster 

import Functions

In [2]:
customers = pd.read_csv('Customer Info Dataset.csv')

In [3]:
transactions = pd.read_csv('Customer Basket Dataset.csv')

In [4]:
prod_mapping = pd.read_excel('Product Mapping Excel File.xlsx')

## customers Dataset

#### 2. Data Cleaning and Feature Extraction and Selection

 **Verifying duplicates**

In [5]:
customers[customers['customer_id'].duplicated() == True]

,customer_id,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude


In [6]:
customers.set_index('customer_id', inplace=True)

In [7]:
customers[customers.duplicated() == True]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,


 **Preliminary Data Analysis**

In [8]:
customers.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,30000.0,1.146700e+00,1.234111,0.000000e+00,0.000000,1.000000,1.000000,10.000000
teens_home,30000.0,5.129333e-01,0.912502,0.000000e+00,0.000000,0.000000,1.000000,9.000000
number_complaints,30000.0,7.643667e-01,0.836341,0.000000e+00,0.000000,1.000000,1.000000,9.000000
distinct_stores_visited,30000.0,6.120767e+00,6.511410,0.000000e+00,2.000000,3.000000,8.000000,21.000000
lifetime_spend_groceries,30000.0,3.978634e+03,5280.641466,0.000000e+00,220.000000,950.000000,5253.000000,18852.000000
lifetime_spend_electronics,30000.0,9.652388e+02,1755.323397,1.058527e+00,45.000000,194.000000,721.000000,6603.000000
typical_hour,30000.0,-inf,NaN,-inf,9.000000,14.000000,20.000000,23.000000
lifetime_spend_vegetables,30000.0,5.642078e+02,481.819086,1.000000e+00,247.000000,425.000000,785.000000,1568.000000
lifetime_spend_nonalcohol_drinks,30000.0,6.287501e+02,496.346400,0.000000e+00,244.000000,425.000000,949.000000,1671.000000
lifetime_spend_alcohol_drinks,30000.0,3.793445e+02,236.206400,0.000000e+00,193.000000,419.000000,537.000000,1048.000000


> Há aqui algos problemas: A variavel typical hour e lifetime spend videogames tem valores infinitos, o que dado o contexto das variaveis em si, condiciona a forma que poderão ser tratadas. 

In [9]:
customers[customers['lifetime_spend_videogames'] == -np.inf]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
1441,Fausto`s Supermarket,male,01/23/1989 11:26 AM,0.0,0.0,0.0,1.0,2.678482,1.095507,0.575646,...,2.613335,34425.0,1.556140,-inf,0.978006,0.199261,2008.0,961221.0,38.866242,-9.110871
3399,Sandy`s Supermarket,female,07/11/1970 12:57 PM,0.0,0.0,0.0,1.0,2.797932,1.141087,0.447940,...,2.623429,36152.0,1.555148,-inf,0.997246,0.201933,2009.0,NaN,38.866366,-9.111207
8086,Eugene`s Supermarket,male,02/29/1956 08:26 PM,0.0,0.0,0.0,1.0,2.802438,1.146242,0.173287,...,2.612483,34777.0,1.530623,-inf,0.951666,0.210895,2010.0,NaN,38.865887,-9.111353
9706,Michele`s Supermarket,female,12/06/1962 02:12 AM,0.0,0.0,0.0,1.0,2.844843,1.184050,0.708303,...,2.615854,35000.0,1.555148,-inf,0.997246,0.200211,2009.0,NaN,38.866273,-9.111174
17805,Robert`s Supermarket,male,09/22/1982 03:36 AM,0.0,0.0,0.0,1.0,2.751432,1.127715,0.599474,...,2.619223,35809.0,1.553152,-inf,1.006338,0.195712,2008.0,NaN,38.866607,-9.111206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689,Justin`s Supermarket,male,03/20/1959 03:59 PM,0.0,0.0,0.0,1.0,2.741558,1.113587,0.621227,...,2.613508,34834.0,1.552650,-inf,0.967800,0.203212,2011.0,NaN,38.866031,-9.111438
16270,Melissa`s Supermarket,female,05/11/1996 05:49 AM,0.0,0.0,0.0,1.0,2.768803,1.175120,0.519860,...,2.619885,34631.0,1.571034,-inf,0.987811,0.198526,2008.0,NaN,38.865962,-9.110773
10262,Francis`s Supermarket,male,11/05/1970 01:58 AM,0.0,0.0,0.0,1.0,2.788190,1.130447,0.693147,...,2.621949,34708.0,1.553152,-inf,0.957160,0.191255,2010.0,NaN,38.866438,-9.111502


In [10]:
customers[customers['typical_hour'] == -np.inf]

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
8592,Charles`s Supermarket,male,01/18/1988 04:13 AM,0.0,0.0,0.0,1.0,2.64414,1.130447,-inf,...,2.615969,35005.0,1.540302,-inf,0.95716,0.196409,2013.0,NaN,38.866685,-9.111366
101,Annie`s Supermarket,female,11/23/1947 10:33 AM,0.0,0.0,0.0,1.0,2.65918,1.133150,-inf,...,2.608573,34702.0,1.553652,-inf,0.91589,0.194451,2009.0,910065.0,38.866048,-9.111718


Parece que são apenas clientes de supermercados com esta particularidade. Vamos então fazer um dataset separado para as pessoas dos supermercados  já que têm um behaviour tão diferente, e proceder a uma análise distinta dos mesmos depois. 

Por enquanto, vou apenas colocar os infinitos como 0 na variavel **lifetime_spend_videogames**, _pelo que o stor me deu a entender essa será uma boa approach_. Para as horas, ainda não sei o que devemos fazer.

In [11]:
customers['lifetime_spend_videogames'] = np.where(customers['lifetime_spend_videogames'] == -np.inf, 0, customers['lifetime_spend_videogames'])

In [12]:
(customers[customers['customer_name'].str.contains('Supermarket')]).describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
teens_home,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
number_complaints,226.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
distinct_stores_visited,226.0,1.000000e+00,0.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000
lifetime_spend_groceries,226.0,2.701444e+00,0.057132,2.423006e+00,2.669260,2.707140,2.741420,2.844843
lifetime_spend_electronics,226.0,1.149498e+00,0.026245,1.058527e+00,1.130447,1.151293,1.165860,1.222587
typical_hour,226.0,-inf,NaN,-inf,0.575646,0.693147,0.722593,0.783874
lifetime_spend_vegetables,226.0,2.124160e+00,0.026242,2.042546e+00,2.105281,2.126912,2.143535,2.183680
lifetime_spend_nonalcohol_drinks,226.0,2.302082e+00,0.012521,2.266444e+00,2.293678,2.303334,2.309999,2.334485
lifetime_spend_alcohol_drinks,226.0,2.704784e+00,0.002484,2.697901e+00,2.703110,2.704669,2.706443,2.712882


> Como podemos ver, os supermercados têm um comportamento totalmente distinto. Também eles tem _typical_hour_ sob o formato de infinitos (ou um formato super mau que não representa horas). 

 **Correcting Missing Values and dtypes;**

In [13]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 4239 to 2570
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   customer_name                            30000 non-null  object 
 1   customer_gender                          30000 non-null  object 
 2   customer_birthdate                       30000 non-null  object 
 3   kids_home                                30000 non-null  float64
 4   teens_home                               30000 non-null  float64
 5   number_complaints                        30000 non-null  float64
 6   distinct_stores_visited                  30000 non-null  float64
 7   lifetime_spend_groceries                 30000 non-null  float64
 8   lifetime_spend_electronics               30000 non-null  float64
 9   typical_hour                             30000 non-null  float64
 10  lifetime_spend_vegetables                300

> Existem missing values somente na variável nr Loyalty card, que poderão se traduzir como "não-aplicável" (a pessoa não tem loyalty card). Não adianta imputar valores aqui, e sendo uma % tão grande de obs, também não ha grande utilidade em recorrer a esta var - mais vale fazer a variável binária. Agora vamos mudar os dtypes.

In [14]:
customers['customer_birthdate'] = pd.to_datetime(customers['customer_birthdate'])
Functions.integer_convert(customers, ['number_complaints', 'kids_home', 'teens_home', 'distinct_stores_visited', 'lifetime_total_distinct_products', 'year_first_transaction'])

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,female,1999-09-10 15:11:00,0,0,0,2,91.0,20.0,22.0,...,53.0,48.0,47.0,105.0,85,0.192154,2017,NaN,38.754263,-9.163397
19330,Donald Robertson,male,1981-06-26 22:58:00,1,0,0,2,197.0,5958.0,21.0,...,86.0,80.0,45.0,1964.0,85,0.098647,2015,NaN,38.777761,-9.156197
2272,Phd. Felicia Bradshaw,female,1987-01-31 09:08:00,1,1,0,20,397.0,20.0,9.0,...,204.0,170.0,46.0,5.0,408,0.497251,2008,NaN,38.785329,-9.127113
25288,Emma Williams,female,1942-06-17 00:02:00,3,4,1,2,5282.0,237.0,12.0,...,1081.0,1068.0,617.0,1078.0,1956,0.206713,2010,NaN,38.744086,-9.156589
12829,Hilda Huerta,female,2000-01-31 11:43:00,0,0,0,2,82.0,22.0,21.0,...,39.0,39.0,48.0,103.0,83,0.200625,2015,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,female,1974-11-01 05:26:00,0,0,1,2,209.0,5474.0,20.0,...,21.0,11.0,57.0,1984.0,112,0.096366,2008,935162.0,38.734446,-9.171911
26447,Curtis Thompson,male,1986-02-21 11:27:00,0,0,0,3,185.0,1066.0,15.0,...,47.0,41.0,50.0,197.0,22,0.118998,2014,NaN,38.767070,-9.144301
182,Richard Carter,male,2001-01-14 05:49:00,0,0,0,2,83.0,23.0,23.0,...,55.0,52.0,51.0,94.0,77,0.199432,2016,953891.0,38.728487,-9.164140


**Separar clientes individuais de supermercados e criar variáveis novas que poderão ser úteis**

Até agora, estas são as variáveis que temos:


_Para clientes individuais_

- **minors_household**: número de menores na household
- **Education**: educação da pessoa (pelo nome)
- **idade**
- **data de nascimento sem horas**: eu acho que isto poderá ser útil para promoções na altura do aniversário e tal :)

_Para supermercados_

_Para ambos_
- **address**: morada retirada com base 
- **total_lifetime_spent**: total gasto
- **loyalty_card**: var binária se tem ou nao loyalty card
- Será que fazemos % de valor gasto em cada tipo de categoria?

**loyalty card**

In [15]:
Functions.binary_encoding(df = customers, col_name = 'loyalty_card', condition = (np.isnan(customers['loyalty_card_number']) == True))
customers.drop('loyalty_card_number', axis=1, inplace=True)

**total lifetime spend**

In [16]:
#total_lifetime_spend
customers['total_lifetime_spend'] = customers[['lifetime_spend_vegetables',
       'lifetime_spend_nonalcohol_drinks', 'lifetime_spend_alcohol_drinks',
       'lifetime_spend_meat', 'lifetime_spend_fish', 'lifetime_spend_hygiene',
       'lifetime_spend_videogames']].sum(axis=1)

**location**

In [ ]:
customers['location'] = customers.apply(lambda row : Functions.get_address(row))

In [ ]:
Functions.encode_address(customers, 'latitude', 'longitude', 'location')

**separar supermarket e individuals**

In [19]:
supermarkets = customers[customers['customer_name'].str.contains('Supermarket')]

In [20]:
individuals = customers[~customers.isin(supermarkets)].dropna(how='all')

In [21]:
individuals.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,29774.0,1.155404,1.234720,0.000000,0.000000,1.000000,1.000000,10.000000
teens_home,29774.0,0.516827,0.914860,0.000000,0.000000,0.000000,1.000000,9.000000
number_complaints,29774.0,0.770169,0.836844,0.000000,0.000000,1.000000,1.000000,9.000000
distinct_stores_visited,29774.0,6.159636,6.520716,0.000000,2.000000,3.000000,8.000000,21.000000
lifetime_spend_groceries,29774.0,4008.813629,5289.228508,0.000000,223.000000,961.000000,5267.000000,18852.000000
lifetime_spend_electronics,29774.0,972.556727,1759.954452,2.000000,46.000000,195.000000,734.750000,6603.000000
typical_hour,29774.0,14.461107,5.509949,0.000000,9.000000,14.000000,20.000000,23.000000
lifetime_spend_vegetables,29774.0,568.474340,481.139698,1.000000,252.000000,429.000000,787.000000,1568.000000
lifetime_spend_nonalcohol_drinks,29774.0,633.505206,495.205308,0.000000,250.000000,431.000000,952.000000,1671.000000
lifetime_spend_alcohol_drinks,29774.0,382.203433,234.802038,0.000000,196.000000,422.000000,537.000000,1048.000000


**minors in household**

In [22]:
individuals['minors_in_household'] = (individuals[['kids_home', 'teens_home']]).sum(axis=1)

**processamento do birthdate**

In [22]:
def process_bithdate(df):
    #birthday
    df['birthday'] = df['customer_birthdate'].dt.day
    #birthmonth
    df['birthmonth'] = df['customer_birthdate'].dt.month
    #birthyear
    df['birthyear'] = df['customer_birthdate'].dt.year

    return df

In [25]:
Functions.process_birthdate(individuals, 'customer_birthdate')

**education**

In [27]:
#extrair educação
individuals['education'] = individuals['customer_name'].apply(lambda row : Functions.extract_education(row))

**encoding das binárias: gender, (...)**

In [29]:

Functions.binary_encoding(individuals, 'customer_gender', condition=(individuals['customer_gender'] == 'male'))

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,loyalty_card_number,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,NaN,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999,0
19330,Donald Robertson,0,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,NaN,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981,0
2272,Phd. Felicia Bradshaw,1,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,NaN,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987,3
25288,Emma Williams,1,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,NaN,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942,0
12829,Hilda Huerta,1,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,NaN,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Phd. Karen Watts,1,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,935162.0,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974,3
26447,Curtis Thompson,0,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,NaN,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986,0
182,Richard Carter,0,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,953891.0,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001,0


In [30]:
#Limpar o nome (tirar a educação)
individuals['customer_name'] = individuals['customer_name'].apply(lambda row: Functions.clean_names(row))

In [31]:
individuals

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,loyalty_card_number,latitude,longitude,loyalty_card,total_lifetime_spend,minors_in_household,birthday,birthmonth,birthyear,education
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1,1999-09-10 15:11:00,0.0,0.0,0.0,2.0,91.0,20.0,22.0,...,NaN,38.754263,-9.163397,0.0,1434.0,0.0,10,9,1999,0
19330,Donald Robertson,0,1981-06-26 22:58:00,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,...,NaN,38.777761,-9.156197,0.0,4064.0,1.0,26,6,1981,0
2272,Felicia Bradshaw,1,1987-01-31 09:08:00,1.0,1.0,0.0,20.0,397.0,20.0,9.0,...,NaN,38.785329,-9.127113,0.0,1336.0,2.0,31,1,1987,3
25288,Emma Williams,1,1942-06-17 00:02:00,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,...,NaN,38.744086,-9.156589,0.0,5676.0,7.0,17,6,1942,0
12829,Hilda Huerta,1,2000-01-31 11:43:00,0.0,0.0,0.0,2.0,82.0,22.0,21.0,...,NaN,38.756024,-9.162970,0.0,1266.0,0.0,31,1,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Karen Watts,1,1974-11-01 05:26:00,0.0,0.0,1.0,2.0,209.0,5474.0,20.0,...,935162.0,38.734446,-9.171911,1.0,4047.0,0.0,1,11,1974,3
26447,Curtis Thompson,0,1986-02-21 11:27:00,0.0,0.0,0.0,3.0,185.0,1066.0,15.0,...,NaN,38.767070,-9.144301,0.0,1165.0,0.0,21,2,1986,0
182,Richard Carter,0,2001-01-14 05:49:00,0.0,0.0,0.0,2.0,83.0,23.0,23.0,...,953891.0,38.728487,-9.164140,1.0,1385.0,0.0,14,1,2001,0


**converter types das variaveis**

In [33]:
def integer_convert(df, cols):
    # Converte as colunas especificadas em cols de float para int
    for col in cols:
        df[col] = df[col].astype(int)
    return df


In [32]:
Functions.integer_convert(individuals, ['number_complaints', 'kids_home', 'teens_home', 'distinct_stores_visited', 'lifetime_total_distinct_products', 'year_first_transaction', 'typical_hour', 'loyalty_card', 'minors_in_household'])


NameError: name 'col' is not defined

In [35]:
individuals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29774 entries, 4239 to 2570
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customer_name                            29774 non-null  object        
 1   customer_gender                          29774 non-null  int64         
 2   customer_birthdate                       29774 non-null  datetime64[ns]
 3   kids_home                                29774 non-null  int64         
 4   teens_home                               29774 non-null  int64         
 5   number_complaints                        29774 non-null  int64         
 6   distinct_stores_visited                  29774 non-null  int64         
 7   lifetime_spend_groceries                 29774 non-null  float64       
 8   lifetime_spend_electronics               29774 non-null  float64       
 9   typical_hour                         

#### 3. Univariate Analysis: Vamos plottar os gráficos de cada variavel separadamente; analisar distribuições e possíveis outliers

Em primeiro lugar separo variaveis: discretas, continuas e categóricas (incluido as binárias). Não sei sinceramente se isto vale a pena ou se está bem feito, mas era só para facilitar os plots.

In [ ]:
categorical = ['loyalty_card', 'customer_name', 'education', 'gender', 'address']
discrete = ['kids_home', 'teens_home',
       'number_complaints', 'distinct_stores_visited']
continuous = ['lifetime_spend_vegetables',
       'lifetime_spend_nonalcohol_drinks', 'lifetime_spend_alcohol_drinks',
       'lifetime_spend_meat', 'lifetime_spend_fish', 'lifetime_spend_hygiene',
       'lifetime_spend_videogames', 'total_lifetime_spent', 'percentage_of_products_bought_promotion']

> **distribuição de observações por localização** (Mapa)

In [ ]:
map_lisbon = folium.Map(location=[38.7223, -9.1393], zoom_start=12)
marker_cluster = MarkerCluster().add_to(map_lisbon)
for index, row in customers.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(marker_cluster)

# Display the map
map_lisbon

#### 4. Multivariate Analysis: Correlação e testes de independencia, vamos também ver se existe alguma relação entre variaveis :)

## Basket Dataset

In [17]:
customers = customers.iloc[:10000, :]

In [18]:
customers

,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,latitude,longitude,loyalty_card,total_lifetime_spend
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,female,1999-09-10 15:11:00,0,0,0,2,91.0,20.0,22.0,...,48.0,47.0,105.0,85,0.192154,2017,38.754263,-9.163397,0,1434.0
19330,Donald Robertson,male,1981-06-26 22:58:00,1,0,0,2,197.0,5958.0,21.0,...,80.0,45.0,1964.0,85,0.098647,2015,38.777761,-9.156197,0,4064.0
2272,Phd. Felicia Bradshaw,female,1987-01-31 09:08:00,1,1,0,20,397.0,20.0,9.0,...,170.0,46.0,5.0,408,0.497251,2008,38.785329,-9.127113,0,1336.0
25288,Emma Williams,female,1942-06-17 00:02:00,3,4,1,2,5282.0,237.0,12.0,...,1068.0,617.0,1078.0,1956,0.206713,2010,38.744086,-9.156589,0,5676.0
12829,Hilda Huerta,female,2000-01-31 11:43:00,0,0,0,2,82.0,22.0,21.0,...,39.0,48.0,103.0,83,0.200625,2015,38.756024,-9.162970,0,1266.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16200,Phd. Penny Clark,female,1991-12-26 11:26:00,0,0,0,3,8133.0,1086.0,19.0,...,709.0,271.0,451.0,359,0.154851,2006,38.764480,-9.093059,0,3402.0
14370,Peggy Juan,female,1974-01-02 01:41:00,1,0,1,8,13834.0,179.0,15.0,...,1394.0,140.0,47.0,4146,0.149721,1999,38.745125,-9.191862,1,5365.0
735,Bsc. Russell Murtha,male,1959-07-23 02:01:00,1,0,0,3,1023.0,51.0,10.0,...,0.0,102.0,64.0,194,0.048963,2012,38.759955,-9.126752,1,1728.0


In [19]:
def process_row(row):
    geolocator = Nominatim(user_agent='my_app')
    full_address = geolocator.reverse(f"{row['latitude']}, {row['longitude']}").address
    full_address = full_address.split(',')
    return full_address[-4]


In [21]:
from multiprocessing import Pool

In [22]:
if __name__ == '__main__':
    pool = Pool(processes=4) # set the number of processes to use
    customers['location'] = pool.map(process_row, customers.itertuples(index=False))
    pool.close()
    pool.join()

customers

PicklingError: Can't pickle <class 'pandas.core.frame.Pandas'>: attribute lookup Pandas on pandas.core.frame failed

In [24]:
from geopy.geocoders import Nominatim
import numpy as np

# Define a function to batch process locations
def batch_reverse_geocode(latitudes, longitudes, batch_size=100):
    geolocator = Nominatim(user_agent='my_app')
    locations = []
    for i in range(0, len(latitudes), batch_size):
        lat_batch = latitudes[i:i+batch_size]
        long_batch = longitudes[i:i+batch_size]
        locations_batch = []
        for lat, lon in zip(lat_batch, long_batch):
            location = geolocator.reverse((lat, lon))
            locations_batch.append(location)
        locations += [loc.address.split(',')[-4] for loc in locations_batch]
    return locations



# Batch process the locations
batch_size = 1000
num_batches = len(customers) // batch_size + 1
locations = []
for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = min(batch_start + batch_size, len(customers))
    batch_locations = batch_reverse_geocode(customers['latitude'][batch_start:batch_end], 
                                             customers['longitude'][batch_start:batch_end])
    locations += batch_locations

# Add the locations to the DataFrame
customers['location'] = locations

KeyboardInterrupt: 

In [25]:
from multiprocessing import Pool
import pandas as pd
from geopy.geocoders import Nominatim

def process_row(row):
    geolocator = Nominatim(user_agent='my_app')
    full_address = geolocator.reverse(f"{row['latitude']}, {row['longitude']}").address
    full_address = full_address.split(',')
    return full_address[-4]

if __name__ == '__main__':
    customers_list = customers.to_dict('records')
    pool = Pool(processes=4) # set the number of processes to use
    results = pool.map(process_row, customers_list)
    pool.close()
    pool.join()

    customers['location'] = pd.Series(results)
